In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install keras

In [ ]:
!pip install nibabel

In [ ]:
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import os


def normalization (train_data,train_y):
    normal_d=np.zeros([100,512,512], dtype = 'float32')
    reshaped_y=np.zeros([100,512,512], dtype = 'float32') 
    for i in range(100):
        d=train_data[:,:,i]
        norm = (d - np.min(d)) / (np.max(d) - np.min(d))
        normal_d[i,:,:]=norm
        reshaped_y[i,:,:]=train_y[:,:,i]
    return normal_d,reshaped_y

def load_tiny_imagenet(data_path):
    example_ni1 = os.path.join(data_path, 'tr_im.nii.gz')
    n1_img = nib.load(example_ni1)
    n1_header = n1_img.header
    x_train = n1_img.get_fdata()#512*512*100
    
    example_ni1 = os.path.join(data_path, 'tr_mask.nii.gz')
    n1_img = nib.load(example_ni1)
    n1_header = n1_img.header
    y_train  = n1_img.get_fdata()#512*512*100
    
    train_d,train_y=normalization(x_train,y_train)
    
    
    return train_d,train_y
# one hot 
def one_hot(y_array):
  
  [m,n,q]=np.shape (y_array)
  y_out=np.zeros((m,n,q,4))
  for num in range (0,m):
    for x in range (0,n):
      for y in range (0,q):
        y_out[num,x,y,int(y_array[num,x,y])]=1
  return y_out
              
x,y=load_tiny_imagenet('/content/drive/My Drive/dataset-covid19/')
plt.show()
x_train=x[:70,:,:]
y_train=y[:70,:,:]
x_test=x[70:,:,:]
y_test=y[70:,:,:]
x_train=np.reshape(x_train, (70, 512,512,1))
y_train=one_hot(y_train)
x_test=np.reshape(x_test, (30, 512,512,1))
y_test2=one_hot(y_test)


In [ ]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras


def unet(pretrained_weights = None,input_size = (512,512,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    #conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(4, 1, activation = 'softmax')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])#binary_crossentropy
    
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

Using TensorFlow backend.


In [ ]:
from keras import backend as K
import keras.callbacks as callbacks
import numpy as np

import warnings


class CustomModelCheckpoint(callbacks.Callback):

    def __init__(self, model, filepath, monitor='val_loss', verbose=0,
                 save_best_only=False, save_weights_only=False,
                 mode='auto', period=1):
        # super(ModelCheckpoint, self).__init__()
        self.monitor = monitor
        self.verbose = verbose
        self.filepath = filepath
        self.save_best_only = save_best_only
        self.save_weights_only = save_weights_only
        self.period = period
        self.epochs_since_last_save = 0
        self.model_for_saving = model

        if mode not in ['auto', 'min', 'max']:
            warnings.warn('ModelCheckpoint mode %s is unknown, '
                          'fallback to auto mode.' % (mode),
                          RuntimeWarning)
            mode = 'auto'

        if mode == 'min':
            self.monitor_op = np.less
            self.best = np.Inf
        elif mode == 'max':
            self.monitor_op = np.greater
            self.best = -np.Inf
        else:
            if 'acc' in self.monitor or self.monitor.startswith('fmeasure'):
                self.monitor_op = np.greater
                self.best = -np.Inf
            else:
                self.monitor_op = np.less
                self.best = np.Inf

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epochs_since_last_save += 1
        if self.epochs_since_last_save >= self.period:
            self.epochs_since_last_save = 0
            filepath = self.filepath.format(epoch=epoch + 1, **logs)
            if self.save_best_only:
                current = logs.get(self.monitor)
                if current is None:
                    warnings.warn('Can save best model only with %s available, '
                                  'skipping.' % (self.monitor), RuntimeWarning)
                else:
                    if self.monitor_op(current, self.best):
                        if self.verbose > 0:
                            print('\nEpoch %05d: %s improved from %0.5f to %0.5f,'
                                  ' saving model to %s'
                                  % (epoch + 1, self.monitor, self.best,
                                     current, filepath))
                        self.best = current
                        if self.save_weights_only:
                            self.model_for_saving.save_weights(filepath, overwrite=True)
                        else:
                            self.model_for_saving.save(filepath, overwrite=True)
                    else:
                        if self.verbose > 0:
                            print('\nEpoch %05d: %s did not improve from %0.5f' %
                                  (epoch + 1, self.monitor, self.best))
            else:
                if self.verbose > 0:
                    print('\nEpoch %05d: saving model to %s' % (epoch + 1, filepath))
                if self.save_weights_only:
                    self.model_for_saving.save_weights(filepath, overwrite=True)
                else:
                    self.model_for_saving.save(filepath, overwrite=True)


                    
                    


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import keras.callbacks as callbacks
def train(model, data):
    numGPU = 1
    epochs = 100
    batch_size = 2
    lr = 0.001
    lr_decay = 0.96
    lam_recon = 0.4
    r = 3
    routings = 3
    shift_fraction = 0.1
    debug = False
    digit = 5
    save_dir = '/content/drive/My Drive/unret_covid/unet_model/'
    t = False
    w = None
    ep_num = 0
    dataset = "my_data"
    appendix=''
    # unpacking the data
    (x_train, y_train), (x_test, y_test) = data
    parallel_model = model
    log = callbacks.CSVLogger(save_dir + '/log' + appendix + '.csv')
    checkpoint1 = CustomModelCheckpoint(model, save_dir + '/best_weights_1' + appendix + '.h5', monitor='val_capsnet_acc', 
                                        save_best_only=False, save_weights_only=True, verbose=1)

    checkpoint2 = CustomModelCheckpoint(model, save_dir + '/best_weights_2' + appendix + '.h5', monitor='val_capsnet_acc',
                                        save_best_only=True, save_weights_only=True, verbose=1)

    lr_decay = callbacks.LearningRateScheduler(schedule=lambda epoch: lr * 0.5**(epoch // 10))

    
    # Begin: Training with data augmentation
    def train_generator(x, y, batch_size, shift_fraction=shift_fraction):
        train_datagen = ImageDataGenerator()  # shift up to 2 pixel for MNIST
        train_datagen.fit(x)
        generator = train_datagen.flow(x, y, batch_size=batch_size, shuffle=True)
        while True:
            x_batch, y_batch = generator.next()
            yield ([x_batch, y_batch])

    parallel_model.fit_generator(generator=train_generator(x_train, y_train, batch_size, shift_fraction),
                                 steps_per_epoch=int(y_train.shape[0] / batch_size), epochs=epochs, 
                                 validation_data=[x_test, y_test], initial_epoch=int(ep_num), 
                                 callbacks=[log, checkpoint1, checkpoint2],
                                 shuffle=True)#lr_decay,

    parallel_model.save(save_dir + '/trained_model_multi_gpu.h5')
    model.save(save_dir + '/trained_model.h5')

    return parallel_model






In [ ]:
save_dir = '/content/drive/My Drive/unret_covid/unet_model/'
os.makedirs(save_dir, exist_ok=True)
try:
    os.system("cp deepcaps.py " + save_dir + "/deepcaps.py")
except:
    print("cp deepcaps.py " + save_dir + "/deepcaps.py")

!ls

drive  sample_data


In [ ]:

model = unet()

train(model=model, data=((x_train, y_train), (x_test, y_test)))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


Epoch 1/100
35/35 [==============================] - 78s 2s/step - loss: 0.4628 - accuracy: 0.9078 - val_loss: 0.2758 - val_accuracy: 0.9320

Epoch 00001: saving model to /content/drive/My Drive/unret_covid/unet_model//best_weights_1.h5


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: Can save best model only with val_capsnet_acc available, skipping.


Epoch 2/100
35/35 [==============================] - 56s 2s/step - loss: 0.2774 - accuracy: 0.9300 - val_loss: 0.2725 - val_accuracy: 0.9320

Epoch 00002: saving model to /content/drive/My Drive/unret_covid/unet_model//best_weights_1.h5
Epoch 3/100
35/35 [==============================] - 56s 2s/step - loss: 0.2556 - accuracy: 0.9300 - val_loss: 0.2362 - val_accuracy: 0.9320

Epoch 00003: saving model to /content/drive/My Drive/unret_covid/unet_model//best_weights_1.h5
Epoch 4/100
35/35 [==============================] - 56s 2s/step - loss: 0.2486 - accuracy: 0.9300 - val_loss: 0.2326 - val_accuracy: 0.9320

Epoch 00004: saving model to /content/drive/My Drive/unret_covid/unet_model//best_weights_1.h5
Epoch 5/100
35/35 [==============================] - 56s 2s/step - loss: 0.2391 - accuracy: 0.9300 - val_loss: 0.2328 - val_accuracy: 0.9320

Epoch 00005: saving model to /content/drive/My Drive/unret_covid/unet_model//best_weights_1.h5
Epoch 6/100
35/35 [==============================] -

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt
save_dir = 'unet_model'




eval_model=unet()
eval_model.load_weights("/content/drive/My Drive/unret_covid/unet_model/best_weights_1.h5")
#for i in range(30):
#  res=eval_model.predict(x_test[i,:,:])
res=eval_model.predict(x_test[:10,:,:,:]) 
  
  





/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


In [ ]:
gt=y_test[:10,:,:]

#plt.imshow(gt)
#plt.show()
#gt=one_hot(gt)
pred = np.around(res)
print(np.shape(res))
print(np.shape(gt))
Y_test.shape[0]

In [ ]:
pred1 = np.argmax(pred,axis = 3)
print(np.shape(pred1))


(10, 512, 512)


In [ ]:
plt.imshow(pred1[1,:,:])
plt.show()
plt.imshow(gt[1,:,:])
plt.show()



In [ ]:
def f(x): 
  return int(x)
f2 = np.vectorize(f) 
gt=f2(gt)
pred2=f2(pred1)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score
gt = np.array(gt, dtype = np.uint8)
pred1 = np.array(pred1, dtype = np.uint8)
#m = MultiLabelBinarizer().fit(gt)

#f1_score(m.transform(gt),m.transform(pred1),average='micro')
f=f1_score(gt[1,:,:],pred1[1,:,:],average='micro')
#print("patient ",str(i),"  =",f)